In [1]:
import os
import pandas as pd
import cx_Oracle
from dotenv import load_dotenv

In [ ]:
SUPPORTER = {
    "BM": "Belügyminisztérium Önkormányzati Államtitkárság (09)",
    "KTM": "Közigazgatási és Területfejlesztési Minisztérium (2023.12.31-ig BMÖÁ) (09)"
}

print(SUPPORTER["BM"])

### Inicializálás

In [2]:
load_dotenv()
username = os.getenv('ORACLE_USERNAME')
password = os.getenv('ORACLE_PASSWORD')
dsn = os.getenv('ORACLE_DSN')
pool = cx_Oracle.SessionPool(user=username, password=password, dsn=dsn, min=2, max=5, increment=1)

input_file = 'TXT Ukrán augusztus.xlsx'
output_file = 'elokeszit_ukran_08.xlsx'

AHT='397451'
KID=f'{AHT}/2024'

def get_data(ksh):
    query = f'''
select 
    onk.ksh, 
    SUBSTR(st.torzs_sz||'-'||st.afakod||'-'||st.mk, 0, 8) as torzsszam,    
    cim.irszam, 
    onk.onev,
    cim.koztnev||' '||cim.kozjell as cim,                       
    cim.hsz1 as hsz, 
    TRIM(rk.megnevezes) as regio
from SUPPORTSYS.t_onkorm_tech onk
left join (select * from SUPPORTSYS.t_stat) st on st.ksh=onk.ksh
left join (select * from SUPPORTSYS.t_cim) cim on cim.ksh=onk.ksh
left join SUPPORTSYS.t_regio rk on rk.regiokod=onk.rk
where onk.ksh='{ksh}'
'''
    connection = pool.acquire()
    cursor = connection.cursor()
    cursor.execute(query)
    data = cursor.fetchall()    
    pool.release(connection)
    return data

### EBR betöltő beolvasás

In [5]:
df = pd.read_excel(f'./data/{input_file}', usecols=[0, 1, 2], names=['KSH', 'ONK', 'AMOUNT'])
df['KSH'] = df['KSH'].astype(str).str.zfill(7)
df['KID'] = KID
df['AHT'] = AHT
df['TORZSSZAM'] = ''
df['IRSZAM'] = ''
df['ONEV'] = ''
df['CIM'] = ''
df['HSZ'] = ''
df['REGIO'] = ''

df.head()

,KSH,ONK,AMOUNT,KID,AHT,TORZSSZAM,IRSZAM,ONEV,CIM,HSZ,REGIO
0,1616647,Fegyvernek Város Önkormányzata,775000,397451/2024,397451,,,,,,
1,2016364,Kisvásárhely Község Önkormányzata,1085000,397451/2024,397451,,,,,,
2,2016081,Kispáli Község Önkormányzata,155000,397451/2024,397451,,,,,,
3,2005704,Pethőhenye Község Önkormányzata,4960000,397451/2024,397451,,,,,,
4,2002097,Gutorfölde Község Önkormányzata,465000,397451/2024,397451,,,,,,


### Szükséges adatok az adatbázisból

In [6]:
column_names = ['TORZSSZAM', 'IRSZAM', 'ONEV', 'CIM', 'HSZ', 'REGIO']

for index, row in df.iterrows():
    ksh_value = row['KSH']
    data = get_data(ksh_value)
    for i, column_name in enumerate(column_names):
        df.at[index, column_name] = data[0][i+1]
df.head()

,KSH,ONK,AMOUNT,KID,AHT,TORZSSZAM,IRSZAM,ONEV,CIM,HSZ,REGIO
0,1616647,Fegyvernek Város Önkormányzata,775000,397451/2024,397451,15736534,5231,Fegyvernek,Szent Erzsébet Út,171,Észak-Alföld
1,2016364,Kisvásárhely Község Önkormányzata,1085000,397451/2024,397451,15437527,8341,Kisvásárhely,Petőfi Utca,7,Nyugat-Dunántúl
2,2016081,Kispáli Község Önkormányzata,155000,397451/2024,397451,15437204,8912,Kispáli,József Attila Utca,40,Nyugat-Dunántúl
3,2005704,Pethőhenye Község Önkormányzata,4960000,397451/2024,397451,15437455,8921,Pethőhenye,Hunyadi János Utca,2,Nyugat-Dunántúl
4,2002097,Gutorfölde Község Önkormányzata,465000,397451/2024,397451,15432041,8951,Gutorfölde,Dózsa Utca,1,Nyugat-Dunántúl


### Bezárás

In [7]:
pool.close()
df.to_excel(f'./data/{output_file}', index=False)